In [1]:
# 이제 필요없는 거는 날릴 때도 됐다.

In [123]:
#한글 폰트 사용
from matplotlib import font_manager,rc
#폰트 경로
font_path = "C:/Users/user/AppData/Local/Microsoft/Windows/Fonts/NanumBarunGothic.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()

In [124]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pymysql
from sklearn import preprocessing
from pycaret.classification import *

In [178]:
# db 연결
def Dataframe():
  conn=pymysql.connect(host='database-1.cani4y1t2hfg.ap-northeast-2.rds.amazonaws.com',port=3306,
                      user='slows14tem',password='!wjdgy0117',db='bigdata', cursorclass=pymysql.cursors.DictCursor)
  cursor = conn.cursor()
  sql = "select * from original"
  cursor.execute(sql)
  result = cursor.fetchall()
  conn.close()
  train = pd.DataFrame(result)
  return train

In [179]:
train = Dataframe()

In [180]:
label_encoder = preprocessing.LabelEncoder()

In [181]:
train['subject'] = label_encoder.fit_transform(train['subject_'])
train['machinery']= label_encoder.fit_transform(train['machinery'])
train['assembly']= label_encoder.fit_transform(train['assembly'])
train['items']= label_encoder.fit_transform(train['items'])
# train['key2']=label_encoder.fit_transform(train['key2'])

In [182]:
columns= ['machinery'  , 'assembly' , 'items', 'subject', 'key2' ]
features= ['machinery'  , 'assembly' , 'items', 'subject']
target= ['key2']

data_Encode 에 해당 컬럼들을 넣는다.

In [183]:
data_Encode= train[columns]
data_Encode

,machinery,assembly,items,subject,key2
0,109,295,4906,953,COOLER
1,114,1201,3733,1673,COOLER
2,114,220,5995,1674,COOLER
3,109,83,700,10,COOLER
4,109,251,4736,524,COOLER
...,...,...,...,...,...
21079,64,488,2839,258,CYLINDER
21080,64,488,2838,256,CYLINDER
21081,0,919,4200,576,BELT
21082,5,1231,1601,237,CYLINDER


In [184]:
from sklearn.model_selection import train_test_split

In [185]:
X_tr, X_val, y_tr, y_val = train_test_split(data_Encode[features], data_Encode[target], stratify = data_Encode[target], test_size= 0.2)

In [186]:
X_tr

,machinery,assembly,items,subject
4512,109,292,4902,582
5900,6,1090,1563,1217
3313,105,1549,2151,1108
1715,150,1285,4713,1318
1347,111,593,2086,1645
...,...,...,...,...
4071,134,1657,222,1270
10606,93,1401,3628,755
17017,109,364,2151,243
7985,62,1199,6120,850


In [187]:
# train, validation
TR = pd.concat([X_tr, y_tr], axis=1)
VAL = pd.concat([X_val, y_val], axis=1)

In [188]:
y_tr

,key2
4512,O-RING
5900,COVER
3313,GASKET
1715,SCREW
1347,GASKET
...,...
4071,SCREW
10606,O-RING
17017,GASKET
7985,GASKET


In [189]:
TR

,machinery,assembly,items,subject,key2
4512,109,292,4902,582,O-RING
5900,6,1090,1563,1217,COVER
3313,105,1549,2151,1108,GASKET
1715,150,1285,4713,1318,SCREW
1347,111,593,2086,1645,GASKET
...,...,...,...,...,...
4071,134,1657,222,1270,SCREW
10606,93,1401,3628,755,O-RING
17017,109,364,2151,243,GASKET
7985,62,1199,6120,850,GASKET


In [200]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
model = random_forest.fit(X_tr, y_tr)
y_pred = random_forest.predict(X_val)
print(f"Accuracy: {accuracy_score(y_val, y_pred):.3f}")

Accuracy: 0.718


In [204]:
save_model(model, './original_model4/model_rf')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['machinery',
                                                             'assembly', 'items',
                                                             'subject'],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='key2',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 f...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                 

환경설정

In [208]:
import joblib

joblib.dump(model, '../original_model4/filename.pkl')


FileNotFoundError: [Errno 2] No such file or directory: '../original_model4/filename.pkl'

In [198]:
y_pred

array(['GASKET', 'BEARING', 'VALVE', ..., 'MOTOR', 'O-RING', 'SCREW'],
      dtype=object)

In [ ]:
clf= setup(data=TR, target='key2', use_gpu=True, categorical_features=['machinery'  , 'assembly' , 'items', 'subject'])
# categorical을 안 주면 label을 숫자로 주니까 쓸 수가 없음
# clf= setup(data=TR, target='key2_Encode', use_gpu=True)


In [122]:
import keras
import numpy
from keras.callbacks import EarlyStopping
es = EarlyStopping()

model = keras.models.Sequential()
model.add(keras.layers.Dense(1, input_shape=(X_tr.shape[1],), activation='sigmoid'))
# model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# it = model.fit(X_tr, y_tr, batch_size=16, epochs=100, callbacks = [es])
it = model.fit(X_tr, y_tr, batch_size=16, epochs=100)

Epoch 1/100


UnimplementedError: Graph execution error:

Detected at node 'categorical_crossentropy/Cast' defined at (most recent call last):
    File "C:\Users\user\miniconda3\envs\pycaret2\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\user\miniconda3\envs\pycaret2\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\user\miniconda3\envs\pycaret2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\user\miniconda3\envs\pycaret2\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\user\miniconda3\envs\pycaret2\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\user\miniconda3\envs\pycaret2\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\user\m

In [111]:
print(X_tr.shape)
print(y_tr.shape)

(14758, 4)
(14758, 1)


BEST 5 출력

In [ ]:
# top5 = compare_models(sort='Accuracy', n_select=10)
# 점심시간 전에 model blending
top5 = compare_models(sort='Accuracy', n_select=5, fold=5)
blended_models = blend_models(top5, fold=5)
finalized_models = finalize_model(blended_models)
save_model(finalized_models, './original_model3/blend')
predictions_blend = predict_model(data=VAL, estimator=finalized_models)
predictions_blend

IntProgress(value=0, description='Processing: ', max=74)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:47:42
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


In [ ]:
'''
https://bigdaheta.tistory.com/53
교수님 말씀은, 니가 평가지표에 대해 제대로 알고 있어야 지도할 게 있다는 건데,
사실 key2, 지금 이 상태에서 분류를 맞추는 데 있어서는 accuracy가 높은 게 장땡아닌가 싶다.
정밀도는, false를 true로 잘못 예측하는 것이 critical할 때 검토할 필요가 있는 지표라고 하는데, 이 경우에도 ridge가 가장 높은 퍼포먼스를 보여줬었다 part1 있을 때는.
지금은 part1 빼버리고 subject를 넣어서 또 모델이 바뀔 것같은데 어떻게 될지는 지켜봐야지..
'''

In [ ]:
lr1 = create_model('lr')
finalize_model(lr1)
save_model(lr1, './original_model3/lr1')
prediction_lr = predict_model(lr1, VAL)
prediction_lr

In [ ]:
knn1 = create_model('knn')
finalize_model(knn1)
save_model(knn1, './original_model3/knn1')
prediction_knn = predict_model(knn1, VAL)
prediction_knn

In [ ]:
nb1 = create_model('nb')
finalize_model(nb1)
save_model(nb1, './original_model3/knn1')
prediction_nb = predict_model(nb1, VAL)
prediction_nb

In [ ]:
dt1 = create_model('dt')
finalize_model(dt1)
save_model(dt1, './original_model3/knn1')
prediction_dt = predict_model(dt1, VAL)
prediction_dt

In [ ]:
svm1 = create_model('svm')
finalize_model(svm1)
save_model(svm1, './original_model3/svm1')
prediction_svm = predict_model(svm1, VAL)
prediction_svm

In [ ]:
rbfsvm1 = create_model('rbfsvm')
finalize_model(rbfsvm1)
save_model(rbfsvm1, './original_model3/rbfsvm1')
prediction_rbfsvm = predict_model(rbfsvm1, VAL)
prediction_rbfsvm

In [ ]:
gpc1 = create_model('gpc')
finalize_model(gpc1)
save_model(gpc1, './original_model3/gpc1')
prediction_gpc = predict_model(gpc1, VAL)
prediction_gpc

In [ ]:
mlp1 = create_model('mlp')
finalize_model(mlp1)
save_model(mlp1, './original_model3/mlp1')
prediction_mlp = predict_model(mlp1, VAL)
prediction_mlp

In [ ]:
ridge1 = create_model('ridge')
finalize_model(ridge1)
save_model(ridge1, './original_model3/ridge1_1')
prediction_ridge = predict_model(ridge1, VAL)
prediction_ridge

In [ ]:
rf1 = create_model('rf')
tuned_rf1 = tune_model(rf1)
final_rf1 = finalize_model(tuned_rf1)
save_model(rf1, './original_model3/rf1')
prediction_rf = predict_model(rf1, VAL)
prediction_rf

In [ ]:
qda1 = create_model('qda')
finalize_model(qda1)
save_model(qda1, './original_model3/rf1')
prediction_qda = predict_model(qda1, VAL)
prediction_qda

In [ ]:
# 21%
ada1 = create_model('ada')
finalize_model(ada1)
save_model(ada1, './original_model3/ada1')
prediction_ada = predict_model(ada1, VAL)
prediction_ada

In [ ]:
gbc1 = create_model('gbc')
finalize_model(gbc1)
save_model(gbc1, './original_model3/gbc1')
prediction_gbc = predict_model(gbc1, VAL)
prediction_gbc

In [ ]:
lda1 = create_model('lda')
finalize_model(lda1)
save_model(lda1, './original_model3/lda1')
prediction_lda = predict_model(lda1, VAL)
prediction_lda

In [ ]:
et1 = create_model('et')
finalize_model(et1)
save_model(et1, './original_model3/et1')
prediction_et = predict_model(et1, VAL)
prediction_et

In [ ]:
lightgbm1 = create_model('lightgbm')
finalize_model(lightgbm1)
save_model(lightgbm1, './original_model3/lightgbm1')
prediction_lightgbm = predict_model(lightgbm1, VAL)
prediction_lightgbm